<a href="https://colab.research.google.com/github/Tennyson66/FraudShield/blob/main/Finetuned_Tinyllama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi

# Get transcript from YouTube
video_id = "HIRG-xUmFuQ"  # Your video ID
transcript = YouTubeTranscriptApi.get_transcript(video_id)
text = " ".join([t['text'] for t in transcript])

# Save transcript to file
with open("video_transcript.txt", "w") as f:
    f.write(text)

print("✅ Transcript saved to video_transcript.txt")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 11.9 MB/s eta 0:00:00
✅ Transcript saved to video_transcript.txt


In [2]:
import json

with open("video_transcript.txt", "r") as f:
    content = f.read()

# Create one fine-tuning example
data = [
    {
        "instruction": "Summarize this YouTube video on cybersecurity",
        "input": content,
        "output": "This video teaches basic cybersecurity awareness, including phishing, secure passwords, and digital safety practices."  # You can change this
    }
]

# Save dataset
with open("yt_dataset.jsonl", "w") as f:
    for d in data:
        f.write(json.dumps(d) + "\n")

print("✅ Dataset saved as yt_dataset.jsonl")


✅ Dataset saved as yt_dataset.jsonl


In [3]:
!pip install transformers datasets accelerate peft bitsandbytes



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [6]:
import json
from transformers import AutoTokenizer

# Load tokenizer (skip if already loaded earlier)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Load the dataset from JSONL file
with open("yt_dataset.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Tokenize each entry
def tokenize(example):
    prompt = f"<|user|>{example['instruction']} {example['input']}<|end|>\n<|assistant|>{example['output']}<|end|>"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_data = [tokenize(item) for item in data]

# Show sample
print("✅ Tokenization successful!")
print(tokenized_data[0])



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

✅ Tokenization successful!
{'input_ids': [1, 529, 29989, 1792, 29989, 29958, 11139, 3034, 675, 445, 14711, 4863, 373, 5094, 495, 8926, 22172, 322, 12853, 9826, 306, 864, 304, 6232, 411, 366, 263, 716, 982, 310, 4417, 2472, 470, 848, 304, 263, 2919, 4086, 1904, 5491, 306, 2048, 17052, 470, 5663, 347, 369, 18765, 287, 12623, 319, 9851, 2820, 2919, 4086, 1904, 2845, 1722, 2752, 470, 451, 1722, 2752, 322, 306, 4240, 1023, 1422, 2924, 310, 5837, 366, 508, 2048, 17052, 541, 9826, 2729, 373, 263, 2009, 306, 2355, 373, 278, 1833, 4863, 310, 920, 366, 508, 788, 278, 470, 1436, 300, 1540, 848, 470, 350, 4037, 848, 373, 263, 2919, 4086, 1904, 306, 2714, 1048, 372, 322, 306, 1284, 714, 920, 591, 508, 1284, 260, 1540, 4359, 738, 1904, 393, 591, 864, 541, 372, 9273, 29915, 29873, 367, 4802, 2723, 29920, 306, 29915, 29885, 773, 871, 1722, 2752, 322, 5087, 7776, 304, 1436, 300, 1540, 365, 3304, 29871, 29906, 2020, 591, 881, 2691, 260, 1540, 263, 1506, 16370, 1904, 591, 871, 2691, 260, 1540, 4733, 746,

In [7]:
from torch.utils.data import Dataset

# Custom dataset class for training
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "attention_mask": self.data[idx]["attention_mask"],
            "labels": self.data[idx]["input_ids"]  # labels = input_ids for causal LM
        }

    def __len__(self):
        return len(self.data)

# Create dataset instance
train_dataset = MyDataset(tokenized_data)

print(f"✅ Dataset ready with {len(train_dataset)} examples")


✅ Dataset ready with 1 examples


In [8]:
!pip install -q peft transformers trl accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [7]:
from datasets import load_dataset

# Load dataset from the YouTube transcript JSONL
dataset = load_dataset("json", data_files="yt_dataset.jsonl", split="train")

# Optional: Truncate to speed up on CPU (remove this if not needed)
dataset = dataset.select(range(min(len(dataset), 10)))

# Convert to train_dataset format
train_dataset = dataset.map(lambda x: {"text": x["text"]})


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

KeyError: 'text'

In [8]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="yt_dataset.jsonl", split="train")

# Print column names
print("✅ Dataset columns:", dataset.column_names)
print("🔁 Sample row:", dataset[0])


✅ Dataset columns: ['instruction', 'input', 'output']
🔁 Sample row: {'instruction': 'Summarize this YouTube video on cybersecurity', 'input': "hello and welcome today I want to share with you a new way of adding information or data to a large language model usually I build rag or retriever augmented generation ABS around large language model either open source or not open source and I built two different kind of ways you can build rag but today based on a request I got on the last video of how you can add the or finetune data or BDF data on a large language model I thought about it and I find out how we can find tune almost any model that we want but it shouldn't be big cuz I'm using only open source and Google cab to finetune Lama 2 why we should fine tune a br trained model we only fine tune models when we have a static or basically a data doesn't change and we want put it inside our large language model and find tun it only on this specific kind of domain so we're not using when the

In [9]:
# Merge instruction, input, and output into a single "text" field
train_dataset = dataset.map(lambda x: {
    "text": f"### Instruction:\n{x['instruction']}\n\n### Input:\n{x['input']}\n\n### Response:\n{x['output']}"
})


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [11]:
from trl import SFTTrainer

# Create trainer (no tokenizer here)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
)

# Assign tokenizer separately
trainer.tokenizer = tokenizer

# Start training
trainer.train()


Adding EOS to train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

{'train_runtime': 244.7721, 'train_samples_per_second': 0.012, 'train_steps_per_second': 0.012, 'train_loss': 3.5601784388224282, 'num_tokens': 3072.0, 'mean_token_accuracy': 0.3398501177628835, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=3.5601784388224282, metrics={'train_runtime': 244.7721, 'train_samples_per_second': 0.012, 'train_steps_per_second': 0.012, 'total_flos': 19088894066688.0, 'train_loss': 3.5601784388224282})

In [12]:
trainer.model.save_pretrained("./finetuned-tinyllama")
tokenizer.save_pretrained("./finetuned-tinyllama")


('./finetuned-tinyllama/tokenizer_config.json',
 './finetuned-tinyllama/special_tokens_map.json',
 './finetuned-tinyllama/chat_template.jinja',
 './finetuned-tinyllama/tokenizer.model',
 './finetuned-tinyllama/added_tokens.json',
 './finetuned-tinyllama/tokenizer.json')

In [13]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load your fine-tuned model
model_path = "./finetuned-tinyllama"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text-generation pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt your fine-tuned model
prompt = "### Instruction:\nSummarize this YouTube video on cybersecurity\n\n### Input:\nhello and welcome today I want to share with you..."
response = pipe(prompt, max_new_tokens=200)[0]['generated_text']

print("🧠 Model Response:\n", response)


🧠 Model Response:
 ### Instruction:
Summarize this YouTube video on cybersecurity

### Input:
hello and welcome today I want to share with you...

### Output:
introduction to cybersecurity, a comprehensive guide on how to protect yourself and your organization from cyber threats.


In [15]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model_path = "./tinyllama-finetuned"  # or the path where your model is saved
pipe = pipeline("text-generation", model=model_path, tokenizer=tokenizer)

# 🔁 Change the instruction and input as needed
instruction = "Summarize this YouTube video on cybersecurity"
input_text = "Today I want to explain some new attack techniques hackers use..."

prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Output:\n"

# Run the model
response = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)

# Display result
print("🧠 Model Response:\n", response[0]['generated_text'][len(prompt):])


ValueError: Could not load model ./tinyllama-finetuned with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 571, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 309, in _wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/d

In [16]:
model.save_pretrained("./tinyllama-finetuned")
tokenizer.save_pretrained("./tinyllama-finetuned")



('./tinyllama-finetuned/tokenizer_config.json',
 './tinyllama-finetuned/special_tokens_map.json',
 './tinyllama-finetuned/chat_template.jinja',
 './tinyllama-finetuned/tokenizer.model',
 './tinyllama-finetuned/added_tokens.json',
 './tinyllama-finetuned/tokenizer.json')

In [1]:
from transformers import pipeline, AutoTokenizer

model_path = "./tinyllama-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)

pipe = pipeline("text-generation", model=model_path, tokenizer=tokenizer)

# 🔁 Prompt input
instruction = "Summarize this YouTube video on cybersecurity"
input_text = "Today I want to explain some new attack techniques hackers use..."

prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Output:\n"

response = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)

print("🧠 Model Response:\n", response[0]['generated_text'][len(prompt):])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


🧠 Model Response:
 Hackers are now using new attack techniques such as ransomware, social engineering, and phishing to infect systems and steal sensitive data. 

### Extra credit:
You can add some examples of these attack techniques and explain how they work to make it more engaging and informative for your listeners.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Replace with the base model path you used for training
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# ✅ Save the fine-tuned model directory
model.save_pretrained("./tinyllama-finetuned")
tokenizer.save_pretrained("./tinyllama-finetuned")



In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = "./tinyllama-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

instruction = "Summarize this YouTube video on cybersecurity"
input_text = "Define ollama"
prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Output:\n"

response = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
print("🧠 Model Response:\n", response[0]['generated_text'][len(prompt):])


Device set to use cpu


🧠 Model Response:
 Ollama is a YouTube video on cybersecurity.
